In [1]:
import pandas as pd

Create FRED Economic Data specific function for formating/cleaning csv files

In [2]:
def fred_clean(name, file):
    """
    name: string input. names the dataframe and title of value header column.
    file: string input. file name of downloaded FRED data set. format as name.csv
    """
    df = str(name) + '_df'
    df = pd.read_csv(r'./data/' + file)
    # rename header
    df.columns = ['date', name]
    # change data type to datetime
    df['date'] = pd.to_datetime(df['date'])
    # remove days from data column. yyyy-mm '-dd'
    df['date'] = df['date'].dt.strftime('%Y-%m')
    # remove duplicate date values (time step = monthly)
    df = df.drop_duplicates('date')
    # set index to date and sort
    df = df.set_index('date', drop = False)
    df = df.sort_index()
    # slice time series data from '2008-01' to '2022-12'
    df = df.loc['2003-01': '2022-12']
    
    # replace values in this dataset that are str('.') 
  
    df = df.replace('.', 0)
        
    # change data type to float in cpi columns
    df[[name]] = df[[name]].astype(float)
    df['date'] = pd.to_datetime(df['date'])
    return df

CREATE: df_11

In [3]:
m1_supply_df = fred_clean('m1_supply', 'M1SL.csv')
gdp_df = fred_clean('gdp', 'GDP.CSV')
fed_funds_rate_df = fred_clean('fed_funds_rate', 'FEDFUNDS.csv')
fed_bal_df = fred_clean('fed_bal', 'RESPPANWW.csv')
gov_debt_df = fred_clean('gov_debt', 'GFDEBTN.csv')

In [4]:
#remove date column (currently indexed)
m1_supply_df = m1_supply_df.drop(['date'], axis=1)
gdp_df = gdp_df.drop(['date'], axis=1)
fed_funds_rate_df = fed_funds_rate_df.drop(['date'], axis=1)
fed_bal_df = fed_bal_df.drop(['date'], axis=1)
gov_debt_df = gov_debt_df.drop(['date'], axis=1)

In [5]:
#merge all data frames to df_11
df_11 = pd.merge(m1_supply_df, gdp_df, how='outer', on = 'date').merge(fed_funds_rate_df, how='outer', on = 'date').merge(fed_bal_df, how='outer', on = 'date').merge(gov_debt_df, how='outer', on = 'date')
#linear interpolation for government_debt_usd quarterly data
df_11 = df_11.interpolate(limit=1, limit_direction='both').fillna(method='ffill')
#create velocity_money = gdp / m1_supply
df_11 ['velocity_money'] = df_11['gdp'] / df_11['m1_supply']
df_11

,m1_supply,gdp,fed_funds_rate,fed_bal,gov_debt,velocity_money
date,,,,,,
2003-01,1227.3,11174.129000,1.24,732202.0,6.460776e+06,9.104644
2003-02,1238.2,11220.341333,1.26,720885.0,6.530558e+06,9.061817
2003-03,1239.3,11266.553667,1.25,723792.0,6.600339e+06,9.091062
2003-04,1250.0,11312.766000,1.26,733626.0,6.670121e+06,9.050213
2003-05,1268.8,11397.400333,1.26,734390.0,6.707854e+06,8.982819
...,...,...,...,...,...,...
2022-08,20472.4,25860.113333,2.33,8874620.0,3.092891e+07,1.263170
2022-09,20283.4,25996.285667,2.56,8822401.0,3.092891e+07,1.281653
2022-10,20100.3,26132.458000,3.08,8759053.0,3.092891e+07,1.300103


In [6]:
# save data frame as df_11
from pathlib import Path  
filepath = Path('./data_frames/df_11.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_11.to_csv(filepath)

CREATE: df_12

In [7]:
#### import data ####
nasdaq100_df = fred_clean('nasdaq100', 'NASDAQ100.csv')
#https://fred.stlouisfed.org/series/NASDAQ100
btc_price_df = fred_clean('btc_price', 'CBBTCUSD.csv')
#https://fred.stlouisfed.org/series/CBBTCUSD
housing_index_df = fred_clean('housing_index', 'USSTHPI.csv')
#https://fred.stlouisfed.org/series/USSTHPI
commod_index_df = fred_clean('commod_index', 'PALLFNFINDEXM.csv')
#https://fred.stlouisfed.org/series/PALLFNFINDEXM
energy_index_df = fred_clean('energy_index', 'PNRGINDEXM.csv')
#https://fred.stlouisfed.org/series/PNRGINDEXM

In [8]:
#drop all other rows after first of month
nasdaq100_df = nasdaq100_df.assign(M=nasdaq100_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
btc_price_df = btc_price_df.assign(M=btc_price_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
housing_index_df = housing_index_df.assign(M=housing_index_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
commod_index_df = commod_index_df.assign(M=commod_index_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)
energy_index_df = energy_index_df.assign(M=energy_index_df["date"].dt.to_period("M")).drop_duplicates("M").drop("M",axis=1)

In [9]:
#remove date column (currently indexed)
nasdaq100_df = nasdaq100_df.drop(['date'], axis=1)
btc_price_df = btc_price_df.drop(['date'], axis=1)
housing_index_df = housing_index_df.drop(['date'], axis=1)
commod_index_df = commod_index_df.drop(['date'], axis=1)
energy_index_df = energy_index_df.drop(['date'], axis=1)

In [10]:
#merge all data frames to df_12
df_12 = pd.merge(nasdaq100_df, btc_price_df, how='outer', on = 'date').merge(housing_index_df, how='outer', on = 'date').merge(commod_index_df, how='outer', on = 'date').merge(energy_index_df, how='outer', on = 'date')
#linear interpolation for government_debt_usd quarterly data
df_12 = df_12.interpolate(limit=1, limit_direction='both').fillna(method='ffill')
df_12

,nasdaq100,btc_price,housing_index,commod_index,energy_index
date,,,,,
2007-02,1830.59,NaN,NaN,116.909043,145.026825
2007-03,1753.45,NaN,380.55,119.617206,148.298187
2007-04,1773.33,NaN,380.55,126.426952,156.852961
2007-05,1873.43,NaN,379.13,126.760827,157.716477
2007-06,1928.26,NaN,377.71,127.077679,162.327163
...,...,...,...,...,...
2006-06,12041.89,16980.55,380.79,117.066399,159.629511
2006-08,12041.89,16980.55,380.79,122.122935,167.897493
2006-09,12041.89,16980.55,380.79,112.400918,145.649770


In [11]:
# save data frame as df_12
from pathlib import Path  
filepath = Path('./data_frames/df_12.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_12.to_csv(filepath)

CREATE: df_13

In [12]:
#### import data ####
cpi_index_df = fred_clean('cpi_index', 'CPIAUCSL.csv')
#https://fred.stlouisfed.org/series/CPIAUCSL
unit_labor_df = fred_clean('unit_labor', 'ULCNFB.csv')
#https://fred.stlouisfed.org/series/ULCNFB
personal_savings_df = fred_clean('personal_savings','PSAVE.csv')
#https://fred.stlouisfed.org/series/PSAVE
personal_expenses_df = fred_clean('personal_expenses', 'PCE.csv')
#https://fred.stlouisfed.org/series/PCE
rent_prim_res_df = fred_clean('rent_prim_res', 'CUUR0000SEHA.csv')
#https://fred.stlouisfed.org/series/CUUR0000SEHA

In [13]:
#remove date column (currently indexed)
cpi_index_df = cpi_index_df.drop(['date'], axis=1)
unit_labor_df = unit_labor_df.drop(['date'], axis=1)
personal_savings_df = personal_savings_df.drop(['date'], axis=1)
personal_expenses_df = personal_expenses_df.drop(['date'], axis=1)
rent_prim_res_df = rent_prim_res_df.drop(['date'], axis=1)

In [14]:
#merge all data frames to df_13
df_13 = pd.merge(cpi_index_df, unit_labor_df, how='outer', on = 'date').merge(personal_savings_df, how='outer', on = 'date').merge(personal_expenses_df, how='outer', on = 'date').merge(rent_prim_res_df, how='outer', on = 'date')
#linear interpolation for government_debt_usd quarterly data
df_13 = df_13.interpolate(limit=1, limit_direction='both').fillna(method='ffill')
df_13

,cpi_index,unit_labor,personal_savings,personal_expenses,rent_prim_res
date,,,,,
2003-01,182.600,90.750000,432.595000,7541.3,203.300
2003-02,183.600,90.873667,437.434333,7548.6,203.700
2003-03,183.900,90.997333,442.273667,7611.5,204.100
2003-04,183.200,91.121000,447.113000,7634.5,204.500
2003-05,182.900,90.911000,461.821000,7650.3,204.900
...,...,...,...,...,...
2022-08,295.320,126.588333,522.721333,17550.9,373.283
2022-09,296.539,126.700667,537.789667,17656.8,376.569
2022-10,297.987,126.813000,552.858000,17797.9,379.436


In [15]:
# save data frame as df_13
from pathlib import Path  
filepath = Path('./data_frames/df_13.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_13.to_csv(filepath)

Merge df_11, df_12 and df_13 to df_1

In [16]:
df_1 = pd.merge(df_11, df_12, how='outer', on = 'date').merge(df_13, how='outer', on = 'date')
df_1

,m1_supply,gdp,fed_funds_rate,fed_bal,gov_debt,velocity_money,nasdaq100,btc_price,housing_index,commod_index,energy_index,cpi_index,unit_labor,personal_savings,personal_expenses,rent_prim_res
date,,,,,,,,,,,,,,,,
2003-01,1227.3,11174.129000,1.24,732202.0,6.460776e+06,9.104644,12041.89,16980.55,279.790000,65.153378,79.454166,182.600,90.750000,432.595000,7541.3,203.300
2003-02,1238.2,11220.341333,1.26,720885.0,6.530558e+06,9.061817,12041.89,16980.55,380.790000,70.229514,90.889264,183.600,90.873667,437.434333,7548.6,203.700
2003-03,1239.3,11266.553667,1.25,723792.0,6.600339e+06,9.091062,12041.89,16980.55,380.790000,65.782768,80.592195,183.900,90.997333,442.273667,7611.5,204.100
2003-04,1250.0,11312.766000,1.26,733626.0,6.670121e+06,9.050213,12041.89,16980.55,283.210000,61.887199,70.847197,183.200,91.121000,447.113000,7634.5,204.500
2003-05,1268.8,11397.400333,1.26,734390.0,6.707854e+06,8.982819,12041.89,16980.55,380.790000,63.065977,72.878676,182.900,90.911000,461.821000,7650.3,204.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08,20472.4,25860.113333,2.33,8874620.0,3.092891e+07,1.263170,12940.78,23250.19,570.698333,241.342146,376.412083,295.320,126.588333,522.721333,17550.9,373.283
2022-09,20283.4,25996.285667,2.56,8822401.0,3.092891e+07,1.281653,12274.62,20112.51,570.698333,219.702441,330.149586,296.539,126.700667,537.789667,17656.8,376.569
2022-10,20100.3,26132.458000,3.08,8759053.0,3.092891e+07,1.300103,11229.73,19308.76,570.698333,191.042783,259.858676,297.987,126.813000,552.858000,17797.9,379.436


In [17]:
# save data frame as df_1
from pathlib import Path  
filepath = Path('./data_frames/df_1.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_1.to_csv(filepath)